In [1]:
# import linecache
# from collections import Counter
# import os
# import tracemalloc
#
# def display_top(snapshot, key_type='lineno', limit=3):
#     snapshot = snapshot.filter_traces((
#         tracemalloc.Filter(False, "<frozen importlib._bootstrap>"),
#         tracemalloc.Filter(False, "<unknown>"),
#     ))
#     top_stats = snapshot.statistics(key_type)
#
#     print("Top %s lines" % limit)
#     for index, stat in enumerate(top_stats[:limit], 1):
#         frame = stat.traceback[0]
#         # replace "/path/to/module/file.py" with "module/file.py"
#         filename = os.sep.join(frame.filename.split(os.sep)[-2:])
#         print("#%s: %s:%s: %.1f KiB"
#               % (index, filename, frame.lineno, stat.size / 1024))
#         line = linecache.getline(frame.filename, frame.lineno).strip()
#         if line:
#             print('    %s' % line)
#
#     other = top_stats[limit:]
#     if other:
#         size = sum(stat.size for stat in other)
#         print("%s other: %.1f KiB" % (len(other), size / 1024))
#     total = sum(stat.size for stat in top_stats)
#     print("Total allocated size: %.1f KiB" % (total / 1024))

# tracemalloc.start()
# counts = Counter()
# experimentation(classification_dataset_names[0])
# snapshot = tracemalloc.take_snapshot()
# display_top(snapshot)

In [2]:
from ex_func import *
from experiment_functions import *
import pandas as pd
from pmlb import fetch_data, classification_dataset_names
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, f_classif
import pickle

In [3]:
def get_feature_type(x, include_binary=False):
    x.dropna(inplace=True)
    if not check_if_all_integers(x):
        return 'continuous'
    else:
        if x.nunique() > 10:
            return 'continuous'
        if include_binary:
            if x.nunique() == 2:
                return 'binary'
        return 'categorical'

def get_target_type(x, include_binary=False):
    x.dropna(inplace=True)
    if x.dtype=='float64':
        return 'continuous'
    elif x.dtype=='int64':
        if include_binary:
            if x.nunique() == 2:
                return 'binary'
        return 'categorical'
    else:
        raise ValueError("Error getting type")

def check_if_all_integers(x):
    "check a pandas.Series is made of all integers."
    return all(float(i).is_integer() for i in x.unique())
def corr_data_for(df):
    TARGET_NAME = 'target'
    feat_names = [col for col in df.columns if col!=TARGET_NAME]
    types = [get_feature_type(df[col], include_binary=True) for col in feat_names]
    col = pd.DataFrame(feat_names,types)
    num_col = col[col.index == 'continuous']
    bin_col = col[col.index == 'binary']
    cat_col = col[col.index == 'categorical']
    cat_col = cat_col[0].tolist()
    dummy_col = pd.get_dummies(data=df, columns=cat_col)
    add_col = dummy_col.shape[1] - df.shape[1]

    if (add_col < df.shape[0] *0.3) & (dummy_col.shape[1] <  df.shape[0]) & (df.shape[0] < 10000) & (df.shape[1] < 100):
        df = dummy_col
        df.columns = df.columns.str.replace('.','_',regex=True)
        y = df['target']
        X = df.loc[:, df.columns != 'target']
        del df
        rows_data, columns_data = X.shape
        print('Dataset Information')
        print('Rows:',rows_data,)
        print('Columns:',columns_data)
        print('Number of classes:',y.nunique())
        print('Continous columns:', len(num_col))
        print('Binary columns:', len(bin_col))
        print('Categorical columns:',len(cat_col))
        print('-------------------------------------------------')
    else:
        del df
        return pd.DataFrame, pd.DataFrame
    return y, X

In [4]:
def split_function(y,X,it):
    sc = StandardScaler()
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = it, stratify=y)
    X_col = X_train.columns
    X_test.name = "X_test"
    X_train.name = "X_train"
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    X_train = pd.DataFrame(X_train,columns=X_col)
    X_test = pd.DataFrame(X_test,columns=X_col)
    return X_train, X_test, y_train, y_test

In [5]:
 classification_dataset_names = classification_dataset_names[30:33]

In [6]:
def experimentation(classification_dataset,iters):
    res_rul = {}
    names = ['Reg-CART','CART','ORT','OCT','ORT-H','OCT-H','ORT+ORT-H','OCT+OCT-H']
    algorithms = ['LN','SVM','NB','KNN']
    pipelines = [LN_pipeline,SVM_pipeline,NB_pipeline,KNN_pipeline]

    df = fetch_data(classification_dataset)
    y, X = corr_data_for(df)
    print('Numer of NANs: ',X.isna().sum().sum() + y.isna().sum())
    del df
    if X.empty:
        return {}

    print(color.BOLD + '\n\n    ----------------------------------------- {} -----------------------------------------'.format(classification_dataset) + color.END)
    for it in range(iters):
        X_train, X_test, y_train, y_test = split_function(y,X,it)
        col_len = len(X_train.columns)
        factors = [0.5,1,1.2,1.4,1.6,1.8,2,2.5,3]

        models, performance = generate_tree(X_train, y_train, X_test, y_test, n_num=1, feat_size=len(X.columns),  max_iter_hy=2,sub_paths=True,depth_grid=range(3,4), depth_grid_hy=range(1,3), complexity_bi = 0.001, complexity_hy=0.001,  Reg_CART=False, ORT=False, ORT_H=False, Clas_CART=True, OCT=True, OCT_H=False)

        for perf,name in zip(performance,names):
            if not not perf:
                res_rul[(classification_dataset,name,it,1)] = sum(perf) / len(perf)

        act_name = []
        act_rules = []
        for model,name in zip(models,names):
            if (all(model)) & (not not model) & (None not in model):
                act_name.append(name)
                act_rules.append(model)

        datasets = gen_train_and_test_features(act_rules ,act_name , X_train, X_test)

        for model in datasets.keys():
            X_train_rules_and_features, X_test_rules_and_features = datasets[model][0]
            X_train_only_rules, X_test_only_rules = datasets[model][1]

            for algorithm,pipeline in zip(algorithms,pipelines):
                res_rul[(classification_dataset,model + f'_{algorithm}_rules',it,'all')] = pipeline(X_train_only_rules, X_test_only_rules, y_train, y_test)
                res_rul[(classification_dataset,model + f'_{algorithm}_rules_and_features',it,'all')] = pipeline(X_train_rules_and_features, X_test_rules_and_features, y_train, y_test)

            for fact in factors:
                if (round(len(X_train_rules_and_features.columns)*fact) <= X_train.shape[0]) & (round(col_len*fact) <= len(X_train_rules_and_features.columns)):
                    min_feat_rule = round(col_len*fact)

                    if (round(col_len*fact) > len(X_train_only_rules.columns)) & (fact != 0.5):
                        len_rule = 1
                        min_rule = len(X_train_only_rules.columns)
                    else:
                        len_rule = fact
                        min_rule = min(round(col_len*fact),len(X_train_only_rules.columns))

                    cols = SelectKBest(k=min_feat_rule).fit(X_train_rules_and_features,y_train).get_feature_names_out()
                    cols_rule = SelectKBest(k=min_rule).fit(X_train_only_rules,y_train).get_feature_names_out()

                else:
                     continue

                for algorithm,pipeline in zip(algorithms,pipelines):
                    res_rul[(classification_dataset,model + f'_{algorithm}_rules',it,len_rule)] = pipeline(X_train_only_rules[cols_rule], X_test_only_rules[cols_rule], y_train, y_test)
                    res_rul[(classification_dataset,model + f'_{algorithm}_rules_and_features',it,fact)] = pipeline(X_train_rules_and_features[cols], X_test_rules_and_features[cols], y_train, y_test)

        for algorithm,pipeline in zip(algorithms,pipelines):
            res_rul[(classification_dataset,algorithm,it,1)] = pipeline(X_train, X_test, y_train, y_test)
        del X_train, X_test
    return res_rul

In [7]:
# for data in classification_dataset_names:
#     data = fetch_data(data)
#     print(data.shape)

In [8]:
orig = {}
for classification_dataset in classification_dataset_names:
    res_rul = experimentation(classification_dataset,1)
    orig.update(res_rul)

Dataset Information
Rows: 106
Columns: 7
Number of classes: 2
Continous columns: 7
Binary columns: 0
Categorical columns: 0
-------------------------------------------------
Numer of NANs:  0


    ----------------------------------------- appendicitis -----------------------------------------


[ Warning: This copy of Interpretable AI software is for academic purposes only and not for commercial use.


Classification CART mean performance:  0.8181818181818181


Classification OCT performance:  0.6818181818181819


Dataset Information
Rows: 690
Columns: 25
Number of classes: 2
Continous columns: 7
Binary columns: 4
Categorical columns: 3
-------------------------------------------------
Numer of NANs:  0


    ----------------------------------------- australian -----------------------------------------
Classification CART mean performance:  0.7898550724637681


Classification OCT performance:  0.782608695652174


Dataset Information
Rows: 202
Columns: 52
Number of classes: 5
Continous columns: 16
Binary columns: 3
Categorical columns: 6
-------------------------------------------------
Numer of NANs:  0


    ----------------------------------------- auto -----------------------------------------
Classification CART mean performance:  0.6341463414634146


Classification OCT performance:  0.7317073170731707




In [11]:
# from joblib import delayed
# from tqdm import tqdm
# res_rul = ProgressParallel(n_jobs=4)(delayed(experimentation)(data) for data in classification_dataset_names)

# result = {}
# for d in res_rul:
#     result.update(d)

0it [00:00, ?it/s]

ERROR: error initializing stdin in uv_pipe_open: inappropriate ioctl for device (ENOTTY -25)
ERROR: error initializing stdin in uv_pipe_open: inappropriate ioctl for device (ENOTTY -25)
ERROR: error initializing stdin in uv_pipe_open: inappropriate ioctl for device (ENOTTY -25)
ERROR: error initializing stdin in uv_pipe_open: inappropriate ioctl for device (ENOTTY -25)


BrokenProcessPool: A task has failed to un-serialize. Please ensure that the arguments of the function are all picklable.

In [10]:
# import os
# files = os.listdir('C:/Users/paulr/PycharmProjects/pythonProject/ORRFA-2/')

In [12]:
# list_of_dfs = []
# for file in files:
#     if file.endswith('pickle'):
#         with open(file, 'rb') as handle:
#             b = pickle.load(handle)
#             # df = pd.concat({k:json_normalize(v, 'scores', ['best']) for k,v in d.items()})
#             # df = df.reset_index(level=1, drop=True).rename_axis('names').reset_index()
# #             list_of_dfs.append(b)
# big_df = pd.concat(list_of_dfs, ignore_index=True)#ignore_index to reset index of big_df
# big_df.head()

In [9]:
k = pd.DataFrame(orig,index=[0])
k = k.stack(level=2).sort_index()
k = k.stack(level=2).sort_index()
k = k.swaplevel(axis=1)
k = k.droplevel(0)

In [10]:
k

CART CART_KNN_rules CART_KNN_rules_and_features CART_LN_rules  \
      appendicitis   appendicitis                appendicitis  appendicitis   
0 0.5          NaN       0.818182                    0.727273      0.818182   
  1       0.818182       0.818182                    0.727273      0.818182   
  1.2          NaN            NaN                    0.727273           NaN   
  1.4          NaN            NaN                    0.772727           NaN   
  1.6          NaN            NaN                    0.818182           NaN   
  all          NaN       0.818182                    0.818182      0.818182   

      CART_LN_rules_and_features CART_NB_rules CART_NB_rules_and_features  \
                    appendicitis  appendicitis               appendicitis   
0 0.5                   0.818182      0.818182                   0.818182   
  1                     0.863636      0.818182                   0.727273   
  1.2                   0.863636           NaN                   0.772727   
  1.4                   0.863636           NaN                   0.772727   
  1.6                   0.772727           NaN                   0.772727   
  all                   0.818182      0.818182                   0.818182   

      CART_SVM_rules CART_SVM_rules_and_features          KNN  ...  \
        appendicitis                appendicitis appendicitis  ...   
0 0.5       0.818182                    0.863636          NaN  ...   
  1         0.818182                    0.772727     0.772727  ...   
  1.2            NaN                    0.863636          NaN  ...   
  1.4            NaN                    0.818182          NaN  ...   
  1.6            NaN                    0.818182          NaN  ...   
  all       0.818182                    0.818182          NaN  ...   

      OCT_LN_rules OCT_LN_rules_and_features OCT_NB_rules  \
              auto                      auto         auto   
0 0.5     0.731707                  0.780488     0.219512   
  1            NaN                       NaN          NaN   
  1.2          NaN                       NaN          NaN   
  1.4          NaN                       NaN          NaN   
  1.6          NaN                       NaN          NaN   
  all     0.731707                  0.804878     0.219512   

      OCT_NB_rules_and_features OCT_SVM_rules OCT_SVM_rules_and_features  ORT  \
                           auto          auto                       auto auto   
0 0.5                  0.414634      0.731707                   0.731707  NaN   
  1                         NaN           NaN                        NaN  NaN   
  1.2                       NaN           NaN                        NaN  NaN   
  1.4                       NaN           NaN                        NaN  NaN   
  1.6                       NaN           NaN                        NaN  NaN   
  all                  0.414634      0.731707                   0.707317  NaN   

      ORT-H Reg-CART       SVM  
       auto     auto      auto  
0 0.5   NaN      NaN       NaN  
  1     NaN      NaN  0.658537  
  1.2   NaN      NaN       NaN  
  1.4   NaN      NaN       NaN  
  1.6   NaN      NaN       NaN  
  all   NaN      NaN       NaN  

[6 rows x 102 columns]

In [37]:
# k.to_csv('result_girdsearch.csv')

In [41]:
k.swaplevel(axis=0).mean(level=0).mean(level=0,axis=1).iloc[1].sort_values(ascending=False)

C:\Users\paulr\AppData\Local\Temp\ipykernel_33684\3857113462.py:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.median(level=1) should use df.groupby(level=1).median().
  k.swaplevel(axis=0).mean(level=0).mean(level=0,axis=1).iloc[1].sort_values(ascending=False)
C:\Users\paulr\AppData\Local\Temp\ipykernel_33684\3857113462.py:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.median(level=1) should use df.groupby(level=1).median().
  k.swaplevel(axis=0).mean(level=0).mean(level=0,axis=1).iloc[1].sort_values(ascending=False)


CART_SVM_rules_and_features         0.801369
Logistic_Regression                 0.799972
CART_LG_rules_and_features          0.795049
Support Vector Machine              0.794241
OCT_SVM_rules_and_features          0.790729
OCT+OCT-H_SVM_rules_and_features    0.790729
OCT_SVM_rules                       0.788537
OCT+OCT-H_SVM_rules                 0.788537
OCT_LG_rules                        0.787764
OCT+OCT-H_LG_rules                  0.787764
CART_KNN_rules_and_features         0.786146
OCT_LG_rules_and_features           0.784099
OCT+OCT-H_LG_rules_and_features     0.784099
OCT+OCT-H_KNN_rules                 0.779443
OCT_KNN_rules                       0.779443
OCT+OCT-H_KNN_rules_and_features    0.774665
OCT_KNN_rules_and_features          0.774665
K-Nearest-Neighbor                  0.771732
OCT                                 0.769067
CART_LG_rules                       0.760673
Naive Bayes                         0.759918
CART_KNN_rules                      0.759871
CART_SVM_r

In [33]:
t=k.mean(level=0,axis=1)
t = t.mean(axis=0)
t.sort_values(ascending = False)

C:\Users\paulr\AppData\Local\Temp\ipykernel_33684\640293622.py:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.median(level=1) should use df.groupby(level=1).median().
  t=k.mean(level=0,axis=1)


Logistic_Regression                 0.790850
OCT                                 0.777091
OCT_LG_rules_and_features           0.770963
OCT+OCT-H_LG_rules_and_features     0.770963
CART_LG_rules_and_features          0.769029
Support Vector Machine              0.768332
OCT_SVM_rules_and_features          0.756744
OCT+OCT-H_SVM_rules_and_features    0.756744
CART_SVM_rules_and_features         0.754992
CART                                0.748986
K-Nearest-Neighbor                  0.740759
OCT_KNN_rules_and_features          0.731947
OCT+OCT-H_KNN_rules_and_features    0.731947
CART_KNN_rules_and_features         0.730521
OCT_LG_rules                        0.715176
OCT+OCT-H_LG_rules                  0.715176
OCT+OCT-H_NB_rules_and_features     0.708839
OCT_NB_rules_and_features           0.708839
OCT+OCT-H_SVM_rules                 0.707052
OCT_SVM_rules                       0.707052
Naive Bayes                         0.706945
CART_NB_rules_and_features          0.705503
OCT_KNN_ru

In [ ]:
y = k.swaplevel(axis=1)
y = y.var(level=0,axis=1)
y = y.mean(axis=0)
good_tests = y[y < 0.01].index
good = list(good_tests)

In [ ]:
vaild_results = k.iloc[:,k.columns.isin(good, level=1)]
vaild_results=vaild_results.mean(level=0,axis=1)
vaild_results.mean(axis=0)

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(nrows = 5, ncols = 4, gridspec_kw = {"hspace": 0.25})
import seaborn as sns
fig.set_size_inches(30, 25)
iteration = 0

for m in range(5):
    for j in range(4):

        dataset = classification_dataset_names[:20][iteration]

        columns = [i for i in k.columns if dataset in i]
        sns.boxplot(k[columns], ax = ax[m, j])

        ax[m, j].set_title(dataset)

        ax[m, j].set_xticklabels(['CART Rules', "OCT Rules", "Logistic Regression", "RuleFit", "ORRFA"])

        iteration += 1



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
fig, ax = plt.subplots()
sns.boxplot(data = performance_by_iter)
fig.set_size_inches(20, 10)
ax.set_xticklabels(performance_by_iter.columns.values)
# ax.set_ylim(0.93, 0.995)
ax.tick_params(rotation = 0, labelsize = 14)
ax.set_ylabel("Accuracy", fontsize = 14)
ax.set_title("Accuracy of Logistic Regression, RuleFit and ORRFA", fontsize = 15)
# ax.set_ylabel()
plt.savefig('Benchmark ORRFA.png')

In [35]:
iters=5
res_rul = {}
sc = StandardScaler()
names = ['Reg-CART','CART','ORT','OCT','ORT-H','OCT-H','ORT+ORT-H','OCT+OCT-H']

for classification_dataset in classification_dataset_names:
    df = fetch_data(classification_dataset)
    df, num_col, bin_col, cat_col = corr_data_for(df)
    if (df.shape[0] > 10000) | (df.shape[1] > 100) | (df.empty):
        continue
    y = df['target']
    X = df.loc[:, df.columns != 'target']

    print(color.BOLD + '\n\n    ----------------------------------------- {} -----------------------------------------'.format(classification_dataset) + color.END)
    rows_data, columns_data = X.shape
    print('Dataset Information')
    print('Rows:',rows_data,)
    print('Columns:',columns_data)
    print('Number of classes:',y.nunique())
    print('Continous columns:', len(num_col))
    print('Binary columns:', len(bin_col))
    print('Categorical columns:',len(cat_col))
    print('-------------------------------------------------')

    for it in range(iters):
        X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = it, stratify=y)
        X_col = X_train.columns
        col_len = len(X_col)
        X_test.name = "X_test"
        X_train.name = "X_train"
        X_train = sc.fit_transform(X_train)
        X_test = sc.transform(X_test)
        X_train = pd.DataFrame(X_train,columns=X_col)
        X_test = pd.DataFrame(X_test,columns=X_col)
        factors = ['all',0.5,1,1.2,1.4,1.6,1.8,2,2.5,3]

        models, performance = generate_tree(X_train, y_train, X_test, y_test, n_num=1, feat_size=len(X.columns),  max_iter_hy=2,sub_paths=True,depth_grid=range(1,7), depth_grid_hy=range(1,3), complexity_bi = 0.001, complexity_hy=0.001,  Reg_CART=False, ORT=False, ORT_H=False, Clas_CART=True, OCT=True, OCT_H=False)
        for perf,name in zip(performance,names):
            if not not perf:
                res_rul[(classification_dataset,name,it,1)] = sum(perf) / len(perf)

        act_name = []
        act_rules = []
        for model,name in zip(models,names):
            if (all(model)) & (not not model) & (None not in model):
                act_name.append(name)
                act_rules.append(model)

        datasets = gen_train_and_test_features(act_rules ,act_name , X_train, X_test)
        for model in datasets.keys():
            X_train_rules_and_features, X_test_rules_and_features = datasets[model][0]
            X_train_only_rules, X_test_only_rules = datasets[model][1]

            for len_c in factors:

                if len_c == 'all':
                    len_rule = 'all'
                    cols = SelectKBest(k='all').fit(X_train_rules_and_features,y_train).get_feature_names_out()
                    X_train_rules_features = X_train_rules_and_features[cols]
                    X_test_rules_features = X_test_rules_and_features[cols]

                    cols_1 = SelectKBest(k='all').fit(X_train_only_rules,y_train).get_feature_names_out()
                    X_train_rules = X_train_only_rules[cols_1]
                    X_test_rules = X_test_only_rules[cols_1]

                elif (round(col_len*len_c) <= X_train.shape[0]) & (round(col_len*len_c) <= len(X_train_rules_and_features.columns)):
                    min_feat_rule = round(col_len*len_c)
                    if (round(col_len*len_c) > len(X_train_only_rules.columns)) & (col_len != 0.5):
                        len_rule = 1
                        min_rule = len(X_train_only_rules.columns)
                    else:
                        len_rule = len_c
                        min_rule = min(round(col_len*len_c),len(X_train_only_rules.columns))


                    cols = SelectKBest(k=min_feat_rule).fit(X_train_rules_and_features,y_train).get_feature_names_out()
                    X_train_rules_features = X_train_rules_and_features[cols]
                    X_test_rules_features = X_test_rules_and_features[cols]

                    cols_1 = SelectKBest(k=min_rule).fit(X_train_only_rules,y_train).get_feature_names_out()
                    X_train_rules = X_train_only_rules[cols_1]
                    X_test_rules = X_test_only_rules[cols_1]

                else:
                     continue

                # Pipeline models
                only_rules_acc_LN = log_regression_pipeline(X_train_rules, X_test_rules, y_train, y_test)
                rules_and_features_acc_LN = log_regression_pipeline(X_train_rules_features, X_test_rules_features, y_train, y_test)
                res_rul[(classification_dataset,model + "_LG_rules",it,len_rule)] = only_rules_acc_LN
                res_rul[(classification_dataset,model + "_LG_rules_and_features",it,len_c)] = rules_and_features_acc_LN

                only_rules_acc_SVM = SVM_pipeline(X_train_rules, X_test_rules, y_train, y_test)
                rules_and_features_acc_SVM = SVM_pipeline(X_train_rules_features, X_test_rules_features, y_train, y_test)
                res_rul[(classification_dataset,model + "_SVM_rules",it,len_rule)] = only_rules_acc_SVM
                res_rul[(classification_dataset,model + "_SVM_rules_and_features",it,len_c)] = rules_and_features_acc_SVM

                only_rules_acc_NB = NB_pipeline(X_train_rules, X_test_rules, y_train, y_test)
                rules_and_features_acc_NB = NB_pipeline(X_train_rules_features, X_test_rules_features, y_train, y_test)
                res_rul[(classification_dataset,model + "_NB_rules",it,len_rule)] = only_rules_acc_NB
                res_rul[(classification_dataset,model + "_NB_rules_and_features",it,len_c)] = rules_and_features_acc_NB

                only_rules_acc_KNN = KNN_pipeline(X_train_rules, X_test_rules, y_train, y_test)
                rules_and_features_acc_KNN = KNN_pipeline(X_train_rules_features, X_test_rules_features, y_train, y_test)
                res_rul[(classification_dataset,model + "_KNN_rules",it,len_rule)] = only_rules_acc_KNN
                res_rul[(classification_dataset,model + "_KNN_rules_and_features",it,len_c)] = rules_and_features_acc_KNN


        res_rul[(classification_dataset,'Logistic_Regression',it,1)] = log_regression_pipeline(X_train, X_test, y_train, y_test)
        res_rul[(classification_dataset,"Support Vector Machine",it,1)] = SVM_pipeline(X_train, X_test, y_train, y_test)
        res_rul[(classification_dataset,"Naive Bayes",it,1)] = NB_pipeline(X_train, X_test, y_train, y_test)
        res_rul[(classification_dataset,"K-Nearest-Neighbor",it,1)] = KNN_pipeline(X_train, X_test, y_train, y_test)



    ----------------------------------------- GAMETES_Epistasis_2_Way_20atts_0.1H_EDM_1_1 -----------------------------------------
Dataset Information
Rows: 1600
Columns: 60
Number of classes: 2
Continous columns: 0
Binary columns: 0
Categorical columns: 20
-------------------------------------------------
Classification CART mean performance:  0.509375


Classification OCT performance:  0.603125


Classification CART mean performance:  0.515625


Classification OCT performance:  0.64375


Classification CART mean performance:  0.553125


Classification OCT performance:  0.621875


Classification CART mean performance:  0.565625


Classification OCT performance:  0.66875


Classification CART mean performance:  0.46562499999999996


Classification OCT performance:  0.65




    ----------------------------------------- GAMETES_Epistasis_2_Way_20atts_0.4H_EDM_1_1 -----------------------------------------
Dataset Information
Rows: 1600
Columns: 56
Number of classes: 2
Continous column

C:\Users\paulr\PycharmProjects\pythonProject\ORRFA-2\venv\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:110: UserWarning: Features [28 29] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
C:\Users\paulr\PycharmProjects\pythonProject\ORRFA-2\venv\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:111: RuntimeWarning: invalid value encountered in divide
  f = msb / msw
C:\Users\paulr\PycharmProjects\pythonProject\ORRFA-2\venv\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:110: UserWarning: Features [28 29] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
C:\Users\paulr\PycharmProjects\pythonProject\ORRFA-2\venv\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:111: RuntimeWarning: invalid value encountered in divide
  f = msb / msw
C:\Users\paulr\PycharmProjects\pythonProject\ORRFA-2\venv\Lib\site-packages\sklearn\

Classification CART mean performance:  0.9695364238410596


Classification OCT performance:  0.976158940397351




C:\Users\paulr\PycharmProjects\pythonProject\ORRFA-2\venv\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:110: UserWarning: Features [28 29] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
C:\Users\paulr\PycharmProjects\pythonProject\ORRFA-2\venv\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:111: RuntimeWarning: invalid value encountered in divide
  f = msb / msw
C:\Users\paulr\PycharmProjects\pythonProject\ORRFA-2\venv\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:110: UserWarning: Features [28 29] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
C:\Users\paulr\PycharmProjects\pythonProject\ORRFA-2\venv\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:111: RuntimeWarning: invalid value encountered in divide
  f = msb / msw
C:\Users\paulr\PycharmProjects\pythonProject\ORRFA-2\venv\Lib\site-packages\sklearn\

Classification CART mean performance:  0.9801324503311258


Classification OCT performance:  0.9774834437086093




...



    ----------------------------------------- breast_cancer_wisconsin -----------------------------------------
Dataset Information
Rows: 569
Columns: 30
Number of classes: 2
Continous columns: 30
Binary columns: 0
Categorical columns: 0
-------------------------------------------------
Classification CART mean performance:  0.9298245614035088


Classification OCT performance:  0.9649122807017544


Classification CART mean performance:  0.9298245614035088


Classification OCT performance:  0.956140350877193


Classification CART mean performance:  0.956140350877193


Classification OCT performance:  0.9473684210526316


Classification CART mean performance:  0.8947368421052632


Classification OCT performance:  0.9385964912280702


Classification CART mean performance:  0.9210526315789473


Classification OCT performance:  0.956140350877193




    ----------------------------------------- breast_w -----------------------------------------
Dataset Information
Rows: 699
Columns: 80
Nu

C:\Users\paulr\PycharmProjects\pythonProject\ORRFA-2\venv\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:110: UserWarning: Features [49] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
C:\Users\paulr\PycharmProjects\pythonProject\ORRFA-2\venv\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:111: RuntimeWarning: invalid value encountered in divide
  f = msb / msw
C:\Users\paulr\PycharmProjects\pythonProject\ORRFA-2\venv\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:110: UserWarning: Features [49] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
C:\Users\paulr\PycharmProjects\pythonProject\ORRFA-2\venv\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:111: RuntimeWarning: invalid value encountered in divide
  f = msb / msw
C:\Users\paulr\PycharmProjects\pythonProject\ORRFA-2\venv\Lib\site-packages\sklearn\featur

Classification CART mean performance:  0.9071428571428571


Classification OCT performance:  0.9


Classification CART mean performance:  0.9357142857142857


Classification OCT performance:  0.9428571428571428




    ----------------------------------------- buggyCrx -----------------------------------------
Dataset Information
Rows: 690
Columns: 34
Number of classes: 2
Continous columns: 7
Binary columns: 3
Categorical columns: 5
-------------------------------------------------
Classification CART mean performance:  0.8405797101449275


Classification OCT performance:  0.8405797101449275


Classification CART mean performance:  0.8478260869565217


Classification OCT performance:  0.8768115942028986




C:\Users\paulr\PycharmProjects\pythonProject\ORRFA-2\venv\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:110: UserWarning: Features [28] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
C:\Users\paulr\PycharmProjects\pythonProject\ORRFA-2\venv\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:111: RuntimeWarning: invalid value encountered in divide
  f = msb / msw
C:\Users\paulr\PycharmProjects\pythonProject\ORRFA-2\venv\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:110: UserWarning: Features [28] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
C:\Users\paulr\PycharmProjects\pythonProject\ORRFA-2\venv\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:111: RuntimeWarning: invalid value encountered in divide
  f = msb / msw
C:\Users\paulr\PycharmProjects\pythonProject\ORRFA-2\venv\Lib\site-packages\sklearn\featur

KeyboardInterrupt: 

In [47]:
res_rul

{('GAMETES_Epistasis_2_Way_20atts_0.1H_EDM_1_1', 'Reg-CART', 0, 1): nan,
 ('GAMETES_Epistasis_2_Way_20atts_0.1H_EDM_1_1', 'CART', 0, 1): 0.521875,
 ('GAMETES_Epistasis_2_Way_20atts_0.1H_EDM_1_1', 'ORT', 0, 1): nan,
 ('GAMETES_Epistasis_2_Way_20atts_0.1H_EDM_1_1', 'OCT', 0, 1): 0.64375,
 ('GAMETES_Epistasis_2_Way_20atts_0.1H_EDM_1_1', 'ORT-H', 0, 1): nan,
 ('GAMETES_Epistasis_2_Way_20atts_0.1H_EDM_1_1', 'OCT-H', 0, 1): nan,
 ('GAMETES_Epistasis_2_Way_20atts_0.1H_EDM_1_1',
  'CART_LG_rules',
  0,
  'all'): 0.521875,
 ('GAMETES_Epistasis_2_Way_20atts_0.1H_EDM_1_1',
  'CART_LG_rules_and_features',
  0,
  'all'): 0.615625,
 ('GAMETES_Epistasis_2_Way_20atts_0.1H_EDM_1_1',
  'CART_SVM_rules',
  0,
  'all'): 0.521875,
 ('GAMETES_Epistasis_2_Way_20atts_0.1H_EDM_1_1',
  'CART_SVM_rules_and_features',
  0,
  'all'): 0.6125,
 ('GAMETES_Epistasis_2_Way_20atts_0.1H_EDM_1_1',
  'CART_NB_rules',
  0,
  'all'): 0.525,
 ('GAMETES_Epistasis_2_Way_20atts_0.1H_EDM_1_1',
  'CART_NB_rules_and_features',
  0,